In [1]:
import os, sys
#Set a specific GPU

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 0 to use

In [2]:
import pandas as pd
import numpy as np
import torch
import transformers

from torch import nn
from transformers import BertModel

from transformers import BertTokenizer
from transformers import BertForTokenClassification
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from torch.nn import MSELoss

import random
import time
import datetime
import re
import pickle
from tqdm.notebook import tqdm

In [3]:
import wandb

wandb.init(project='test_jlk_nlp_TC_new_zip')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jun171. Use `wandb login --relogin` to force relogin


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [5]:
main_loc = '../'

with open(main_loc+'data/tag_data_flat_lst_zip.pkl','rb') as f:
    dt_flat = pickle.load(f)

In [6]:
dt_flat

,txt_list,tag_list
0,"[suspect, adverse, reaction, report, 2, ., dat...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[of, administration, (, exclude, those, used, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[following, event, with, respect, to, ce, ##f,...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[suspect, adverse, reaction, report, 2, ., dat...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[by, mis, (, medical, information, service, ),...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...
5748,"[##cm, ., path, ##ological, diagnosis, :, pres...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5749,"[suspect, adverse, reaction, report, 2, ., dat...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5750,"[description, unknown, 24, ##a, ., name, and, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5751,"[no, con, ##com, ##itan, ##t, medication, info...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [7]:
train_df,test_df = train_test_split(dt_flat, random_state=1, test_size=0.3)

# train_df,test_df = train_test_split(train_df, random_state=0, test_size=0.3)

In [8]:
lst = []
for i in range(len(dt_flat)):
    lst.append(len(dt_flat.iloc[1]['txt_list']))
min(lst)

509

In [9]:
len(train_df)

4027

In [10]:
root_address = main_loc+'model/'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [12]:
def pad_label(label):
    zero_vec = [0 for _ in range(50)]
    max_length = 512
    pad_lab = label.copy()
    pad_lab.insert(0,zero_vec)   #start token
    pad_lab.append(zero_vec)     #end token
    
    pad_len = max_length-len(pad_lab)
    
    for i in range(pad_len):
        pad_lab.append(zero_vec)
    
    if len(pad_lab)!=512:
        print('weird',len(pad_lab))    
    
    pad_lab  = pad_lab[:512]
    
    
    return pad_lab

In [13]:
def att_mask(input):
    attention_masks = []

    for seq in input:
        seq_mask = []
        for s in seq:
            if s>0:
                seq_mask.append(float(1))
            else:
                seq_mask.append(float(0))
            
        attention_masks.append(seq_mask)
    return attention_masks

In [14]:
def encode_load_data(df):
    encode_inputs = []
    labels = []

    for n in tqdm(range(len(df))):
        row = df.iloc[n]
        
        encode_input = tokenizer.encode(row['txt_list'],padding ='max_length', truncation = True,max_length = 512)
        encode_inputs.append(encode_input)
        labels.append(pad_label(row['tag_list']))
        
    return encode_inputs,labels

In [15]:
train_df

,txt_list,tag_list
3155,"[##86, ##4, ), ), [, 100, ##20, ##86, ##4, ], ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4652,"[causal, ##ity, as, per, reporter, (, drug, /,...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5096,"[##rine, ,, and, 1000, ml, of, normal, saline,...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2300,"[sensation, /, chest, tight, ##ness, ,, ana, #...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4060,"[2020, ;, 18, ., 7, (, unspecified, units, ), ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...
905,"[decreased, met, ##form, ##in, dos, ##ing, fre...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5192,"[suspect, adverse, reaction, report, i, ., rea...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3980,"[##t, ., ., ., ), product, -, reaction, level,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
235,"[suspect, adverse, reaction, report, i, ., rea...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [16]:
train_inputs_o, train_labels_o = encode_load_data(train_df)
test_inputs_o, test_labels_o = encode_load_data(test_df)

  0%|          | 0/4027 [00:00<?, ?it/s]

  0%|          | 0/1726 [00:00<?, ?it/s]

In [17]:
train_attention_masks = att_mask(train_inputs_o)
test_attention_masks = att_mask(test_inputs_o)


In [18]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_inputs_o, train_labels_o, random_state=0, test_size=0.1)
train_masks, validation_masks = train_test_split(train_attention_masks, random_state=0, test_size=0.1)

In [19]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

test_inputs = torch.tensor(test_inputs_o)
test_labels = torch.tensor(test_labels_o)
test_masks = torch.tensor(test_attention_masks)

In [20]:
BATCH_SIZE = 24 #24

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [21]:
class BertforMultitag(nn.Module):

    def __init__(self, dropout=0.1, tag_size = 50):
        self.dropout = dropout
        self.num_labels = tag_size
        
        
        super(BertforMultitag, self).__init__()

        self.bert = BertModel.from_pretrained('bert-large-uncased')
        
        # self.bert = BertModel(BertModel.from_pretrained('bert-large-uncased').config)
        # for param in self.bert.parameters():
        #     param.requires_grad = False
            
        self.dropout = nn.Dropout(self.dropout)
        self.linear1 = nn.Linear(1024, self.num_labels)
        self.relu1 = nn.LeakyReLU()
        self.linear2 = nn.Linear(self.num_labels, self.num_labels)
        self.relu2 = nn.LeakyReLU()
        
        nn.init.kaiming_normal_(self.linear1.weight)
        nn.init.kaiming_normal_(self.linear2.weight)
        
        
    def forward(self, input_id, attention_mask,labels = None):
        global op
        outputs = self.bert(input_ids= input_id, attention_mask=attention_mask,return_dict=False)
        op = outputs
        sequence_output = outputs[0]
        dropout_output = self.dropout(sequence_output)
        linear_output1 = self.linear1(dropout_output)
        relu_output1 = self.relu1(linear_output1)
        linear_output2 = self.linear2(relu_output1)
        logits = self.relu2(linear_output2)
        
        loss = None
        if labels is not None:
            loss_fct = MSELoss()
            loss = loss_fct(logits.view(-1, self.num_labels).type(torch.FloatTensor), labels.view(-1, self.num_labels).type(torch.FloatTensor))
        
        rst = {'loss':loss,'logits':logits}

        return rst

In [22]:
def file_check(root_address,key):
    for file in os.listdir(root_address):
        p_file = re.match(key+'_(\d{3,4})[.]pt$',file)
        if p_file:
            return p_file
    return None

In [23]:
# model = BertforMultitag()

# for param in model.bert.parameters():
#     param.requires_grad = True
    
# model.to(device.type)

In [24]:
file_match = file_check(root_address,'model_max_acc_n_zip')
if file_match == None:
    print('please check the name of the model')
    raise ValueError

model = torch.load(root_address+file_match.group(0))
for param in model.bert.parameters():
    param.requires_grad = True
    
model.to(device.type)

BertforMultitag(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [25]:
optimizer = AdamW(model.parameters(),
                  lr =3e-6,   #3e-6
                  eps = 1e-12
                )

epochs = 10000


total_steps = len(train_dataloader) * epochs



scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps,
                                            num_cycles=1000
                                            )

In [26]:
def num_corrections(pred,label):
    ans = 0
    for ind in pred:
        if ind in label:
            ans+=1
            
    return ans

def flat_accuracy(outputs, labels,th = 0.1):
    
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    tot_len = 0
    tot_ans = 0
    
    for batch_o,batch_l in zip(outputs,labels):
        for out,lab in zip(batch_o,batch_l):
            out_ind = np.where(out>th)[0]
            lab_ind = np.where(lab>th)[0]
            max_len = max(len(out_ind),len(lab_ind))
            mat_num = num_corrections(out_ind,lab_ind)
            
            tot_len+=max_len
            tot_ans+=mat_num
        
    acc = tot_ans/tot_len
    return acc

def removeNone(lst):
    if 0 in lst:
        lst = np.delete(lst,np.where(lst==0))
    return lst

def notNone_accuracy(outputs, labels,th = 0.1):
    
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    tot_len = 0
    tot_ans = 0
    
    for batch_o,batch_l in zip(outputs,labels):
        for out,lab in zip(batch_o,batch_l):
            out_ind = removeNone(np.where(out>th)[0])
            lab_ind = removeNone(np.where(lab>th)[0])
            
            max_len = max(len(out_ind),len(lab_ind))
            if max_len ==0:
                continue
            mat_num = num_corrections(out_ind,lab_ind)
            
            tot_len+=max_len
            tot_ans+=mat_num
    if tot_len ==0:
        return None        
    acc = tot_ans/tot_len
    return acc


def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [27]:
def check_prev_prec(root_address,prec,model,key):
    p_file = file_check(root_address,key)
    if p_file:
        p_p = int(p_file.group(1))
        if prec*100>p_p+1:
            torch.save(model,root_address+key+f'_{prec*100:.0f}.pt')
            os.remove(root_address+p_file.group(0))
            print(f'New {key} has been saved')
        else:
            print(f'Unsaved: low accuracy')
        print(f'Previous accuracy: {p_p/100}')
        print(f'Present accuracy: {prec:.2f}')
    else:
        torch.save(model,root_address+key+f'_{prec*100:.0f}.pt')
        print(f'{key} has been saved')  

In [28]:
def check_logit(outputs):
    rst = []
    for i in range(1,len(outputs['logits'][0])-1):
        if outputs['logits'][0][i].max()<0.01:
            rst.append(i)
    if len(rst)>=2:
        print(rst)

In [29]:
seed_val = 0
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

max_acc = 0
max_acc_n = 0

for epoch_i in range(0, epochs):
    print(f'\nTraining...\n======== Epoch {epoch_i + 1} / {epochs} ========')
    
    t0 = time.time()
    total_loss = 0
    prev_loss = 0
    prev_step = 0
    
    model.train()
        
    for step, batch in enumerate(tqdm(train_dataloader)):

        # if step % 100 == 0 and not step == 0:
        #     elapsed = format_time(time.time() - t0)
        #     step_loss = (total_loss-prev_loss) / (step-prev_step)
        #     print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        #     print(f'  Average loss =  {step_loss}')
        #     print(f"  Last learning rate: {scheduler.get_last_lr()[0]:.3}")
        #     # print(max(outputs['logits'][0][0].detach().cpu().numpy()))
        #     # print(outputs['logits'][0][0].detach().cpu().numpy())        
            
        #     prev_loss = total_loss
        #     prev_step = step            

        # if step % 3000 == 0 and not step == 0:
        #     torch.save(model,root_address+f'model_sv_{epoch_i}_on.pt')
        #     print(f'model_sv_{epoch_i}_on has been saved')
  

        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        
        optimizer.zero_grad()

        # Forward         
        outputs = model(b_input_ids, 
                        attention_mask = b_input_mask,
                        labels = b_labels)
        
        
        loss = outputs['loss']
        # check_logit(outputs)      
        # if torch.flatten(outputs['logits']).max()>2:
        #     print(torch.flatten(outputs['logits']).max())
        #     print(torch.flatten(outputs['logits']).min())

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print(f"  Average training loss: {avg_train_loss:.3}")
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
    print(f"  Last learning rate: {scheduler.get_last_lr()[0]:.3}")
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")


    t0 = time.time()

    torch.save(model,root_address+f'model_sv_zip_{epoch_i}.pt')
    print(f'model_sv_{epoch_i} has been saved')

    if epoch_i >=2:
        os.remove(root_address+f'model_sv_zip_{epoch_i-2}.pt')

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy, eval_accuracy_n = 0, 0, 0
    nb_eval_steps,nb_eval_steps_n= 0,0

    for batch in tqdm(validation_dataloader):
        # if step % 100 == 0 and not step == 0:
        #     elapsed = format_time(time.time() - t0)
        #     print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():     

            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
        
        preds = outputs['logits']
        
        tmp_eval_accuracy= flat_accuracy(preds,b_labels)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        
        tmp_eval_accuracy_n= notNone_accuracy(preds,b_labels)
        if tmp_eval_accuracy_n != None:
            eval_accuracy_n += tmp_eval_accuracy_n
            nb_eval_steps_n += 1        

    acc_all = eval_accuracy/nb_eval_steps*100
    acc_all_n = eval_accuracy_n/nb_eval_steps_n*100

    print(f"  Validation \n  Accuracy_all: {acc_all:.2f}, Max_Accuracy: {max_acc:.2f}")
    print(f"  NonZero_Accuracy_all: {acc_all_n:.2f}, Max_Accuracy: {max_acc_n:.2f}")
    print("  Time took: {:}".format(format_time(time.time() - t0)))    
    print("")
    
    wandb.log({"val_accuracy": acc_all,"val_NonZero_accuracy": acc_all_n,"loss": avg_train_loss, "learning_rate": scheduler.get_last_lr()[0]}, step=epoch_i)    
    
    if max_acc < acc_all:
        print(f"Accuracy: {acc_all:.2f} is higher than previous maximum accuracy: {max_acc:.2f}")
        max_acc = acc_all
        check_prev_prec(root_address,max_acc,model,'model_max_acc_zip')
        # torch.save(model,root_address+f'model_max_acc_zip_{acc_all*100:.0f}.pt')
        # print(f'New model_max_acc has been saved')        

    if max_acc_n < acc_all_n:
        print(f"NonZero Accuracy: {acc_all_n:.2f} is higher than previous maximum accuracy: {max_acc_n:.2f}")
        max_acc_n = acc_all_n
        check_prev_prec(root_address,max_acc_n,model,'model_max_acc_n_zip')
        # torch.save(model,root_address+f'model_max_acc_n_zip_{acc_all_n*100:.0f}.pt')
        # print(f'New model_max_acc_n has been saved')  

    # ========================================
    #                  Test
    # ========================================
    if epoch_i%10 ==0 and epoch_i!=0:
        print("")
        print("Running Test...")

        t0 = time.time()

        model.eval()

        eval_loss, eval_accuracy, eval_accuracy_n = 0, 0, 0
        nb_eval_steps,nb_eval_steps_n= 0,0

        for batch in tqdm(test_dataloader):

            batch = tuple(t.to(device) for t in batch)

            b_input_ids, b_input_mask, b_labels = batch

            with torch.no_grad():     

                outputs = model(b_input_ids, 
                                attention_mask=b_input_mask)
            
            preds = outputs['logits']
            
            tmp_eval_accuracy= flat_accuracy(preds,b_labels)
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1
            
            tmp_eval_accuracy_n= notNone_accuracy(preds,b_labels)
            if tmp_eval_accuracy_n != None:
                eval_accuracy_n += tmp_eval_accuracy_n
                nb_eval_steps_n += 1

        acc_all = eval_accuracy/nb_eval_steps*100
        acc_all_n = eval_accuracy_n/nb_eval_steps_n*100

        acc_all = eval_accuracy/nb_eval_steps*100
        wandb.log({"test_accuracy": acc_all}, step=epoch_i)
        wandb.log({"test_NonZero_accuracy": acc_all_n}, step=epoch_i)
        
        print(f"  Test Accuracy_all: {acc_all:.2f}")
        print(f"  Test NonZeroAccuracy_all: {acc_all_n:.2f}")
        print("  Time took: {:}".format(format_time(time.time() - t0)))    
        print("")

    
print("Training complete!")
torch.save(model,root_address+'model_zip_fin.pt')


Training...
======== Epoch 1 / 10000 ========


  0%|          | 0/151 [00:00<?, ?it/s]


  Average training loss: 4.37e-05
  Training epoch took: 0:04:16
  Last learning rate: 2.93e-06

Running Validation...
model_sv_0 has been saved


  0%|          | 0/17 [00:00<?, ?it/s]

  Validation 
  Accuracy_all: 95.31, Max_Accuracy: 0.00
  NonZero_Accuracy_all: 90.56, Max_Accuracy: 0.00
  Time took: 0:00:16

Accuracy: 95.31 is higher than previous maximum accuracy: 0.00
Unsaved: low accuracy
Previous accuracy: 95.4
Present accuracy: 95.31
NonZero Accuracy: 90.56 is higher than previous maximum accuracy: 0.00
Unsaved: low accuracy
Previous accuracy: 90.75
Present accuracy: 90.56

Training...
======== Epoch 2 / 10000 ========


...


  Average training loss: 4.69e-05
  Training epoch took: 0:04:16
  Last learning rate: 1.5e-06

Running Validation...
model_sv_24 has been saved


  0%|          | 0/17 [00:00<?, ?it/s]

  Validation 
  Accuracy_all: 94.99, Max_Accuracy: 95.35
  NonZero_Accuracy_all: 89.85, Max_Accuracy: 90.64
  Time took: 0:00:16


Training...
======== Epoch 26 / 10000 ========


  0%|          | 0/151 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# ========================================
#                  Test
# ========================================

print("")
print("Running Test...")

t0 = time.time()

model.eval()

eval_loss, eval_accuracy, eval_accuracy_n = 0, 0, 0
nb_eval_steps,nb_eval_steps_n= 0,0

for step, batch in enumerate(tqdm(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     

        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask)
    
    preds = outputs['logits']
    
    tmp_eval_accuracy= flat_accuracy(preds,b_labels)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
    
    tmp_eval_accuracy_n= notNone_accuracy(preds,b_labels)
    if tmp_eval_accuracy_n != None:
        eval_accuracy_n += tmp_eval_accuracy_n
        nb_eval_steps_n += 1

acc_all = eval_accuracy/nb_eval_steps*100
acc_all_n = eval_accuracy_n/nb_eval_steps_n*100

print(f"  Test Accuracy_all: {acc_all:.2f}")
print(f"  Test NonZeroAccuracy_all: {acc_all_n:.2f}")
print("  Time took: {:}".format(format_time(time.time() - t0)))    
print("")